# F1-score аналитической модели

Определим точность работы модели

In [1]:
import pandas as pd

from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
from tqdm import tqdm
from sklearn.metrics import f1_score

Задаём модель, которая будет определать тональность комментариев.

In [2]:
dostoevsky_model = FastTextSocialNetworkModel(tokenizer=RegexTokenizer())

Загружаем данные для проверки модели

In [3]:
df_neg = pd.read_csv('negative.csv', sep=';')
df_pos = pd.read_csv('positive.csv', sep=';')

Объединяем негативные и позитивные комментарии в один датафрейм.

In [4]:
df_res = pd.concat([df_neg, df_pos])
df_res.reset_index(drop=True, inplace=True) 

Модель покажет себя более чем 226 тысячах комментариев.

In [12]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   text         226834 non-null  object
 1   is_positive  226834 non-null  int64 
 2   predictions  226834 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 5.2+ MB


Создаём поле, куда запишем наши предсказания

In [5]:
df_res['predictions'] = None

Теперь прогоним каждый комментарий через моделель

In [6]:
for row in tqdm(df_res.itertuples()):
    text = []
    text.append(row.text)
    results = dostoevsky_model.predict(text, k=5)
    del results[0]['skip']
    del results[0]['speech']
    results[0][0] = results[0].pop("neutral")
    results[0][-1] = results[0].pop("negative")
    results[0][1] = results[0].pop('positive')
    ordered_dict = dict(sorted(results[0].items(), reverse=True, key=lambda x: x[1]))
    first_key = next(iter(ordered_dict))
    df_res.iloc[row.Index, 2] = first_key

226834it [08:25, 448.78it/s]


Датасет был размечен на позитивные и негативные комментарии. Поэтому заменим нейтральные предсказания на позитивные.

In [7]:
def sentiment_pos(sentiment):
    if sentiment == 0:
        return 1
    else:
        return sentiment

In [8]:
def sentiment_neg(sentiment):
    if sentiment == -1:
        return 0
    else:
        return sentiment

In [9]:
df_res['predictions'] = df_res['predictions'].apply(sentiment_pos)

In [10]:
df_res['predictions'] = df_res['predictions'].apply(sentiment_neg)

In [11]:
f1_score(df_res['is_positive'], df_res['predictions'])

0.8089100218907422

**Вывод:** \
Метрика f1-score выбранной модели показала себя очень хорошо и оказалась равна 0.81.